Prepare Dataset for Training

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Load dataset
df = pd.read_csv("student_data.csv") 

# One-Hot Encode Project Domains
project_cols = ["Project 1", "Project 2", "Project 3", "Project 4"]
encoder = OneHotEncoder(sparse_output=False)
project_encoded = encoder.fit_transform(df[project_cols])
project_encoded_df = pd.DataFrame(project_encoded, columns=encoder.get_feature_names_out(project_cols))



# Define thresholds
strong_threshold = 85
weak_threshold = 65

# Encode Strong & Weak Subjects
subjects = ['Operating System', 'DSA', 'Frontend', 'Backend', 'Machine Learning', 'Data Analytics']

# Function to determine strong and weak subjects for a student
def categorize_subjects(row):
    strong = [subject for subject in subjects if row[subject] >= strong_threshold]
    weak = [subject for subject in subjects if row[subject] <= weak_threshold]
    return pd.Series([', '.join(strong), ', '.join(weak)])

# Apply function to dataset
df[['Strong Subjects', 'Weak Subjects']] = df.apply(categorize_subjects, axis=1)

# Initialize binary columns for strong/weak subjects
for subject in subjects:
    df[f"Strong_{subject}"] = df['Strong Subjects'].apply(lambda x: 1 if subject in x else 0)
    df[f"Weak_{subject}"] = df['Weak Subjects'].apply(lambda x: 1 if subject in x else 0)

# Normalize Marks
scaler = MinMaxScaler()
df[subjects] = scaler.fit_transform(df[subjects])

# Combine all features
df_final = pd.concat([df, project_encoded_df], axis=1)

# Drop unnecessary columns
df_final.drop(columns=project_cols + ['Strong Subjects', 'Weak Subjects'], inplace=True)

# Save the processed dataset
df_final.to_csv("processed_student_data.csv", index=False)

print("Feature Engineering Completed ✅")


Feature Engineering Completed ✅


Train a Classification Model

In [2]:
df = df.drop(columns=["Student ID"], errors="ignore")
print(df.columns)



Index(['Operating System', 'DSA', 'Frontend', 'Backend', 'Machine Learning',
       'Data Analytics', 'Attendance (Operating System)', 'Attendance (DSA)',
       'Attendance (Frontend)', 'Attendance (Backend)',
       'Attendance (Machine Learning)', 'Attendance (Data Analytics)',
       'Project 1', 'Project 2', 'Project 3', 'Project 4', 'Strong Subjects',
       'Weak Subjects', 'Strong_Operating System', 'Weak_Operating System',
       'Strong_DSA', 'Weak_DSA', 'Strong_Frontend', 'Weak_Frontend',
       'Strong_Backend', 'Weak_Backend', 'Strong_Machine Learning',
       'Weak_Machine Learning', 'Strong_Data Analytics',
       'Weak_Data Analytics'],
      dtype='object')


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load processed dataset
df = pd.read_csv("processed_student_data.csv")
df = df.drop(columns=["Student ID"], errors="ignore")

# Drop Student ID (irrelevant for learning)
X = df.drop(columns=["Student ID"], errors="ignore")

# Select project columns
project_cols = [col for col in df.columns if col.startswith("Project")]

# Generate 'Most Common Project Domain' dynamically
df["Most Common Project Domain"] = df[project_cols].idxmax(axis=1).str.replace("Project \d+_", "", regex=True)

# Encode categorical target variable
label_encoder = LabelEncoder()
df["Most Common Project Domain"] = label_encoder.fit_transform(df["Most Common Project Domain"])

# Create new feature: Count of projects per domain
for domain in ["AI", "Cybersecurity", "Data Science", "Game Development", "Machine Learning", "Robotics", "Web Development"]:
    df[f"Project_Count_{domain}"] = df.filter(like=domain).sum(axis=1)

# Use these as input features
X = df.drop(columns=project_cols, errors="ignore")

# Define Target Variable
y = df["Most Common Project Domain"]

# Split into Train and Test Set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

# Train a Random Forest Model
model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# Predict on Test Set
y_pred = model.predict(X_test)

# Model Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}% ✅")

# Decode Predictions (Optional)
decoded_labels = label_encoder.inverse_transform(y_pred)
print("Predicted Categories:", decoded_labels[:10])


Model Accuracy: 99.40% ✅
Predicted Categories: ['Data Science' 'Game Development' 'Cybersecurity' 'Game Development'
 'Data Science' 'AI' 'Data Science' 'Robotics' 'Web Development'
 'Data Science']


In [4]:
from sklearn.model_selection import cross_val_score

# Perform 5-Fold Cross Validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

print(f"Cross-Validation Accuracy Scores: {cv_scores}")
print(f"Mean Accuracy: {cv_scores.mean() * 100:.2f}%")


Cross-Validation Accuracy Scores: [1.    0.995 0.98  1.    0.995]
Mean Accuracy: 99.40%


Ensuring Robustness

In [5]:
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print(feature_importances.head(10))


                           Feature  Importance
24      Most Common Project Domain    0.340521
26     Project_Count_Cybersecurity    0.038010
6    Attendance (Operating System)    0.034542
29  Project_Count_Machine Learning    0.034154
27      Project_Count_Data Science    0.033872
28  Project_Count_Game Development    0.033399
2                         Frontend    0.032209
5                   Data Analytics    0.031875
8            Attendance (Frontend)    0.031320
9             Attendance (Backend)    0.031257


In [6]:
print(f"Number of Features in X: {len(X.columns)}")
print(f"Number of Features in Model: {len(model.feature_importances_)}")

model.fit(X_train, y_train)

Number of Features in X: 32
Number of Features in Model: 32


RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load processed dataset
df = pd.read_csv("processed_student_data.csv")

df = df.drop(columns=["Student ID"], errors="ignore")

# Drop Student ID (irrelevant for learning)
X = df.drop(columns=["Student ID"], errors="ignore")

# Select project columns
project_cols = [col for col in df.columns if col.startswith("Project")]

# Generate 'Most Common Project Domain' dynamically
df["Most Common Project Domain"] = df[project_cols].idxmax(axis=1).str.replace("Project \d+_", "", regex=True)

# Encode categorical target variable
label_encoder = LabelEncoder()
df["Most Common Project Domain"] = label_encoder.fit_transform(df["Most Common Project Domain"])

# Create new feature: Count of projects per domain
for domain in ["AI", "Cybersecurity", "Data Science", "Game Development", "Machine Learning", "Robotics", "Web Development"]:
    df[f"Project_Count_{domain}"] = df.filter(like=domain).sum(axis=1)

# Use these as input features
X = df.drop(columns=project_cols, errors="ignore")

# Define Target Variable
y = df["Most Common Project Domain"]

# Split into Train and Test Set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42, stratify=y)

# Train a Random Forest Model
model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# Predict on Test Set
y_pred = model.predict(X_test)

# Model Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}% ✅")

# Decode Predictions (Optional)
decoded_labels = label_encoder.inverse_transform(y_pred)
print("Predicted Categories:", decoded_labels[:10])


Model Accuracy: 85.56% ✅
Predicted Categories: ['Data Science' 'AI' 'Game Development' 'Machine Learning'
 'Game Development' 'Web Development' 'Cybersecurity' 'Data Science'
 'Machine Learning' 'Game Development']


In [8]:
# Predict on Test Set
y_pred = model.predict(X_test)

# Evaluate Model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 85.56%


In [9]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model, X_train, y_train, cv=5)
print(f"Cross-Validation Accuracy Scores: {cv_scores}")
print(f"Mean Accuracy: {cv_scores.mean() * 100:.2f}%")


Cross-Validation Accuracy Scores: [0.65 0.75 0.75 0.65 0.75]
Mean Accuracy: 71.00%


In [10]:
import pandas as pd

# Get Feature Importance
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print(feature_importances.head(10))  # Show top 10 important features


                           Feature  Importance
24      Most Common Project Domain    0.228737
4                 Machine Learning    0.047395
9             Attendance (Backend)    0.044993
29  Project_Count_Machine Learning    0.043451
8            Attendance (Frontend)    0.043320
3                          Backend    0.040606
5                   Data Analytics    0.039297
0                 Operating System    0.038164
2                         Frontend    0.038066
25                Project_Count_AI    0.036474
